In [1]:
import langchain
import langchain_community
import langchain_google_genai
import dotenv
import langchain_pinecone
import pinecone
import os


In [2]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from dotenv import load_dotenv
from langchain_pinecone import PineconeVectorStore
load_dotenv()
api_key=os.getenv("PINECONE_API_KEY")

In [3]:
from langchain_community.document_loaders import UnstructuredURLLoader

In [4]:
urls=['https://www.livemint.com/economy/budget-2024-key-highlights-live-updates-nirmala-sitharaman-infrastructure-defence-income-tax-modi-budget-23-july-11721654502862.html',
        'https://cleartax.in/s/budget-2024-highlights',
        'https://www.hindustantimes.com/budget',
        'https://economictimes.indiatimes.com/news/economy/policy/budget-2024-highlights-india-nirmala-sitharaman-capex-fiscal-deficit-tax-slab-key-announcement-in-union-budget-2024-25/articleshow/111942707.cms?from=md']
loader= UnstructuredURLLoader(urls=urls)

In [5]:
data= loader.load()

The data is loaded

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs= text_splitter.split_documents(data)

print(len(docs))

173


In [7]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings= HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vector = embeddings.embed_query("hello")

vector[:5]

C:\Users\chhan\AppData\Local\Temp\ipykernel_22760\3338247974.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings= HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


[-0.0627717599272728,
 0.05495879426598549,
 0.052164845168590546,
 0.0857900083065033,
 -0.0827488973736763]

In [8]:
from langchain.vectorstores import Pinecone

In [9]:
import os
PINECONE_INDEX_NAME="demo-project"

docsearch= Pinecone.from_existing_index(index_name=PINECONE_INDEX_NAME,embedding=embeddings)
print("Index successfully loaded")

Index successfully loaded


In [14]:
vectorstore_from_docs= PineconeVectorStore.from_documents(
    docs,
    index_name=PINECONE_INDEX_NAME,
    embedding=embeddings
)

In [10]:
docsearch=PineconeVectorStore.from_existing_index(PINECONE_INDEX_NAME,embeddings)

In [11]:
retriever= docsearch.as_retriever(search_type="similarity", search_kwargs={"k":10})


In [33]:
load_dotenv()
groq_api_key=os.getenv("groq_api_key")


Using llama3 model through chatgroq

In [29]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature=0, 
    groq_api_key=groq_api_key,
    model_name="llama3-8b-8192"
      )

In [30]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt=(
    "You are an assistant for question-answer task."
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer , say you don't know."
    "Use three sentences maximum and keep the answer concise"
   "\n \n"
   "{context}" )

prompt= ChatPromptTemplate.from_messages(
    [
    ("system", system_prompt),
    ("human","{input}"),
    ]
)

In [31]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain= create_retrieval_chain(retriever, question_answer_chain)

In [32]:
response=rag_chain.invoke({"input":"what is the budget plan ?"})
print(response["answer"])

The budget plan for 2024 includes several key highlights, such as:

* Rs 2 lakh crore package announced for 5 schemes
* Large-scale clusters for vegetable production will be developed near major consumption centers
* Financial support for setting up Nucleus Breeding Centres for Shrimp Broodstocks
* Allocation of Rs 3 lakh crore for women-centric schemes
* Rs 10 lakh crore allocated to provide 1 crore houses for the urban poor and middle class
* Establishment of 100 branches of India Post Payments Bank in the North East region
* Introduction of a rental housing scheme for the working class
* Amendment to the Insolvency and Bankruptcy Code (IBC) and enhancement of tribunals
* Digitalization of land records and fiscal support to incentivize states to introduce land and labour reforms.

The budget also focuses on employment, skilling, MSMEs, and the middle class, with a priority on generating employment for all.
